In [4]:
!pip install influxdb-client

  Using cached https://files.pythonhosted.org/packages/db/12/ccf0940e7081560a6d677bdb7488312dbda5d0b44ba8016a7a16248f3a10/influxdb_client-1.7.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2c/da/626910cf8aca7ed2d5b34355eee8aeaaeb6ddd4e16f98d00a9e2ddad3a08/ciso8601-2.1.3.tar.gz
  Created wheel for ciso8601: filename=ciso8601-2.1.3-cp37-cp37m-win_amd64.whl size=14537 sha256=bafb9ae01cbbf956358abeb9d62ae11eacb3728e341c7a9a423af4e5c331a62f
  Stored in directory: C:\Users\Michiel\AppData\Local\pip\Cache\wheels\eb\32\e4\13bdaf7e245f82667b21e0cfb03d21224691a47fa9f9bc80a6
Successfully built ciso8601


In [5]:
import influxdb_client

In [8]:
from influxdb_client import InfluxDBClient, Point, Dialect
from influxdb_client.client.write_api import SYNCHRONOUS

client = InfluxDBClient(url="http://localhost:9999", token="my-token", org="my-org")

write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

"""
Prepare data
"""

_point1 = Point("my_measurement").tag("location", "Prague").field("temperature", 25.3)
_point2 = Point("my_measurement").tag("location", "New York").field("temperature", 24.3)

write_api.write(bucket="my-bucket", record=[_point1, _point2])

"""
Query: using Pandas DataFrame
"""
data_frame = query_api.query_data_frame('from(bucket:"my-bucket") '
                                        '|> range(start: -10m) '
                                        '|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value") '
                                        '|> keep(columns: ["location", "temperature"])')
print(data_frame.to_string())

"""
Close client
"""
client.__del__()

MaxRetryError: HTTPConnectionPool(host='localhost', port=9999): Max retries exceeded with url: /api/v2/write?org=my-org&bucket=my-bucket&precision=ns (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000233066DC348>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

## Flux
Flux is a query language for  

### Example
query='''
from(bucket:"my-bucket")
        |> range(start: 0, stop: now())
        |> filter(fn: (r) => r._measurement == "financial-analysis")
        |> filter(fn: (r) => r.symbol == "AAPL")
        |> filter(fn: (r) => r._field == "close")
        |> drop(columns: ["_start", "result", "_stop", "table", "_field","_measurement"])
        |> rename(columns: {_value: "close"})
'''